In [1]:
import warnings
warnings.filterwarnings('ignore')

### Model1: TF-IDF + ML Classifier

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier


from transformers import PreTrainedTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


from datasets import Dataset
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import sys
import gc

In [3]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
train = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv", sep=',')

In [4]:
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)
train.head()

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False


In [5]:
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)

In [6]:
LOWERCASE = False
VOCAB_SIZE = 14_000_000

In [7]:
# Creating Byte-Pair Encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))


# Adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFC()] + [normalizers.Lowercase()] 
    if LOWERCASE else []
)


raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(
    vocab_size=VOCAB_SIZE, 
    special_tokens=special_tokens
)


# Creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])

def train_corp_iter():
    """
    A generator function for iterating over a dataset in chunks.
    """    
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

# Training from iterator REMEMBER it's training on test set...
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token  = "[UNK]",
    pad_token  = "[PAD]",
    cls_token  = "[CLS]",
    sep_token  = "[SEP]",
    mask_token = "[MASK]",
)



# Tokenize test set with new tokenizer
tokenized_texts_test = []
for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))


# Tokenize train set
tokenized_texts_train = []
for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/44868 [00:00<?, ?it/s]

In [8]:
print(tokenized_texts_test[1])
print()
print(tokenized_texts_test[2])

['ĠBbb', 'Ġccc', 'Ġddd', '.']

['ĠCCC', 'Ġddd', 'Ġeee', '.']


In [9]:
def dummy(text):
    """
    A dummy function to use as tokenizer for TfidfVectorizer. 
    It returns the text as it is since we already tokenized it.
    """
    return text



# Fitting TfidfVectoizer on test set
vectorizer = TfidfVectorizer(
    ngram_range   = (3, 5), 
    lowercase     = False, 
    sublinear_tf  = True, 
    analyzer      = 'word',
    tokenizer     = dummy,
    preprocessor  = dummy,
    token_pattern = None, 
    strip_accents ='unicode')


vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_
print(vocab)


# Here we fit our vectorizer on train set but this time we use vocabulary from test fit.
vectorizer = TfidfVectorizer(
    ngram_range    = (3, 5), 
    lowercase      = False, 
    sublinear_tf   = True, 
    vocabulary     = vocab,
    analyzer       = 'word',
    tokenizer      = dummy,
    preprocessor   = dummy,
    token_pattern  = None, 
    strip_accents  ='unicode')

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)
del raw_tokenizer, trainer, tokenized_texts_test, tokenized_texts_train, vectorizer
gc.collect()

{'ĠAaa Ġbbb Ġccc': 0, 'Ġbbb Ġccc .': 6, 'ĠAaa Ġbbb Ġccc .': 1, 'ĠBbb Ġccc Ġddd': 2, 'Ġccc Ġddd .': 7, 'ĠBbb Ġccc Ġddd .': 3, 'ĠCCC Ġddd Ġeee': 4, 'Ġddd Ġeee .': 8, 'ĠCCC Ġddd Ġeee .': 5}


58

In [10]:
y_train_label = train['label'].values

In [11]:
if len(test.text.values) <= 5:
    sub.to_csv('submission.csv', index=False)
else:
    clf = MultinomialNB(alpha=0.0225)
    
    sgd_model = SGDClassifier(
        max_iter     = 9000, 
        tol          = 1e-4, 
        random_state = 6743,
        loss         = "modified_huber"
    ) 
    
    p={
        'verbose'          : -1,
        'n_iter'           : 3000,
        'colsample_bytree' : 0.7800,
        'colsample_bynode' : 0.8000, 
        'random_state'     : 6743,
        'metric'           : 'auc',
        'objective'        : 'cross_entropy',
        'learning_rate'    : 0.00581909898961407, 
      }
    lgb=LGBMClassifier(**p)
    
    
    cat = CatBoostClassifier(
        iterations        = 3000,
        verbose           = 0,
        subsample         = 0.35,
        random_seed       = 6543,
        allow_const_label = True,
        loss_function     = 'CrossEntropy',
        learning_rate     = 0.005599066836106983,
    )
    
    
    ensemble = VotingClassifier(
        estimators = [('mnb', clf),
                      ('sgd', sgd_model),
                      ('lgb', lgb), 
                      ('cat', cat)],
        weights    = [0.1, 0.31, 0.28, 0.67], 
        voting     = 'soft', 
        n_jobs     = -1
    )
    
    ensemble.fit(tf_train, y_train_label)
    gc.collect()
    
    final_preds = ensemble.predict_proba(tf_test)[:,1]
    sub['generated'] = final_preds
    #sub.to_csv('submission.csv', index=False)
    #sub.head()
    del clf, sgd_model, lgb, cat, ensemble
    gc.collect()

In [12]:
del tf_train, tf_test
gc.collect()

0

### Model2: Deberta

In [13]:
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from transformers import AutoModel, AutoTokenizer

In [14]:
input_path = '/kaggle/input/llm-detect-ai-generated-text'
test_data = pd.read_csv(f'{input_path}/test_essays.csv')
sub2 = pd.read_csv(f'{input_path}/sample_submission.csv')

In [15]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class ClassifierModel(nn.Module):
    def __init__(self, checkpoint):
        super(ClassifierModel, self).__init__()
        self.bert_model = AutoModel.from_pretrained(checkpoint)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 1)
        self.pool = MeanPooling()
        
    def encode(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert_model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=False
        )

        encoder_layer = outputs.last_hidden_state
        embeddings = self.pool(encoder_layer, attention_mask)

        return embeddings
        
    def forward(self, x):
        bert_output = self.encode(**x)
        x = self.dropout(bert_output)
        x = self.classifier(x)
        return x

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
model_path = '/kaggle/input/detect-ai-generated-text-deberta-model'
tokenizer = AutoTokenizer.from_pretrained(f'{model_path}/bert-tokenizer')
#model_name = "/kaggle/input/deberta-v3-small/deberta-v3-small"  
#tokenizer = AutoTokenizer.from_pretrained(model_name)
model = torch.load(f'{model_path}/epoch2_valid_loss_0.11764985185027832_auc_0.9871818302651691_model.bin',
                  map_location=torch.device(device))

In [18]:
class EssayDataSet(Dataset):
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data.iloc[idx]

test_data = EssayDataSet(f'/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')


def collate_fn(batch_samples):
    return tokenizer([batch_sample['text'] for batch_sample in batch_samples], padding=True,truncation=True, max_length=512,
                 return_tensors="pt")

test_dataloader = DataLoader(test_data, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [19]:
model.eval()

y_pred = []
with torch.no_grad():
    for X in test_dataloader:
        X = X.to(device)
        pred = model(X)
        y_pred.extend(F.sigmoid(pred).cpu().numpy().flatten())
        
sub2['generated'] = y_pred

### Rank Ensemble

In [20]:
sub["generated"] = sub["generated"].rank(method='min')
sub2["generated"] = sub["generated"].rank(method='min')

In [21]:
sub_df = pd.concat([sub, sub2])
sub_df = sub_df.groupby(['id'])['generated'].mean().reset_index()
sub_df.to_csv('submission.csv', index=False)